In [1]:
!pip install pytesseract pillow pdf2image tk

In [2]:
!apt-get install -y poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5 [186 kB]
Fetched 186 kB in 2s (123 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 123623 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.5_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.5) ...
Setting up poppler-utils (22.02.0-2ubuntu0.5) ...
Processing triggers for man-db (2.10.2-1) ...


In [3]:
!apt-get install -y tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 2s (1,952 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 123653 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-

In [4]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
from google.colab import files

# Carica il PDF tramite interfaccia di Colab
uploaded = files.upload()

# Estrai il nome del file caricato
for filename in uploaded.keys():
    pdf = filename

Saving TEST - Invoice.pdf to TEST - Invoice.pdf


In [6]:
from google.colab import files
import pytesseract
from PIL import Image
from pdf2image import convert_from_path
import os
import pandas as pd
import re

pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'

# Funzione per caricare il PDF tramite Colab
def select_pdf_file():
    print("Carica il file PDF")
    uploaded = files.upload()
    for filename in uploaded.keys():
        return filename  # Restituisce il nome del file caricato

# Funzione per estrarre il testo da un'immagine usando pytesseract
def extract_text_from_image(image_path):
    # Imposta il percorso di Tesseract manualmente se non è nel PATH
    img = Image.open(image_path)

    # Configura Tesseract con PSM 4
    custom_config = f'--psm 4'
    text = pytesseract.image_to_string(img, config=custom_config)
    return text

# Funzione per correggere il prezzo totale, eliminando caratteri non necessari
def correggi_totale(prezzo_totale):
    # Rimuovi caratteri come "1)" o altre cifre aggiuntive dopo il prezzo
    prezzo_totale = re.sub(r'[^\d.,]', '', prezzo_totale)  # Rimuove qualsiasi carattere non numerico o virgola/punto

    # Se ci sono più di due cifre decimali (es. "178.808"), tronca a due cifre decimali
    if re.match(r'\d+\.\d{3}', prezzo_totale):  # Se ci sono tre cifre decimali
        prezzo_totale = prezzo_totale[:-1]  # Troncamento per portarlo a due decimali (es. "178.808" -> "178.80")

    return prezzo_totale

def formatta_valori(valore):
    if isinstance(valore, float):
        return f"{valore:.2f}".replace('.', ',')
    return valore

# Funzione per strutturare i dati estratti in colonne
def structure_extracted_data(extracted_text):
    data_processati = []  # Lista per i dati processati
    data_non_processati = []  # Lista per le righe non processate
    data_non_riconosciuti = []  # Lista per le righe con re errata
    processa_dati = False  # Flag per iniziare e fermare il processo di estrazione

    for page_text in extracted_text:
        lines = page_text.split("\n")
        for line in lines:
            line = line.strip()
            # print(f"Riga estratta: {line}")  # Stampa per verificare cosa sta leggendo esattamente l'OCR

            # Inizia a processare dopo questa intestazione
            if ('Total (EUR)' in line or
                'Your reference' in line):
                processa_dati = True
                continue  # Salta questa riga, ma imposta il flag per iniziare a processare

            # Smetti di processare se trovi una di queste righe
            if ('Please note that third party payments' in line or
                'Pallet fee' in line or
                'Transport' in line):
                processa_dati = False
                continue  # Salta questa riga e smetti di processare le righe successive

            # Processa solo se il flag è True
            if processa_dati:

                pattern = re.compile(r'([\w\-]+)\s+.*?(\d+[,\.]?\d*)\s+liter\s+(\d+)\s+bottles?\s+(\d+[,\.]\d+)')

                # Esegui il regex per catturare Reference, Volume, Quantity, Price
                # match = re.match(r'([\w\-]+)\s+(.*?)\s+(\d+\.\d*\s+\w+)\s+(\d+\s+\w+)\s+(\d+\.\d+)', line)
                match = pattern.match(line)

                try:

                    # Estrarre i gruppi dal regex
                    anker_ref = match.group(1)  # Cattura l'Anker Ref
                    volume = match.group(2).replace(',', '.').strip()  # Converte la virgola in punto
                    quantity = match.group(3).strip()  # Quantità
                    price = float(match.group(4).replace(',', '.'))  # Prezzo per bottiglia
                    total = price * int(quantity)  # Calcola il totale
                    total_str = f"{total:.2f}".replace('.', ',')  # Converte il totale in stringa con virgola come separatore

                    # Determinare la descrizione per differenza
                    end_of_ref = match.end(1)
                    start_of_volume = match.start(2)
                    description = line[end_of_ref:start_of_volume].strip()

                    # Aggiungi i dati processati alla lista
                    data_processati.append([anker_ref, description, volume, quantity, price, total_str])

                except Exception as e:
                    # In caso di errore (ad esempio, una riga spezzata), aggiungi ai non processati
                    # print(f"Errore nella riga: {line}, errore: {e}")
                    data_non_riconosciuti.append(line)
            else:
                data_non_processati.append(line)

    return data_processati, data_non_processati, data_non_riconosciuti

#####################################################################################################################
# INIZIO CODICE MAIN METHOD #########################################################################################
#####################################################################################################################

# 1. Selezionare il PDF con una dialog box
pdf_path = pdf


if pdf_path:

    # Lista per memorizzare i percorsi delle immagini generate
    image_paths = []

    # 2. Convertire il PDF in immagini
    images = convert_from_path(pdf_path, dpi=450)

    # 3. Applicare l'OCR su ogni immagine
    extracted_text = []

    for i, image in enumerate(images):

        image_path = f'/content/immagine_{i+1}.png'
        image.save(image_path, 'PNG')

        # Aggiungi il percorso dell'immagine alla lista
        image_paths.append(image_path)

        # Apre l'immagine per usarla con Tesseract
        img = Image.open(image_path)

        # Estraggo il testo dalle immagini usando tesseract
        text = extract_text_from_image(image_path)

        # Aggiunge il testo alla lista creando una stringa unica
        extracted_text.append(text)

        # Chiudi l'immagine per rilasciare le risorse
        img.close()

    # print(extracted_text)

    # 4. Strutturare i dati
    structured_data, unstructured_data, error_data = structure_extracted_data(extracted_text)

    # 5. Creare un DataFrame per organizzare i dati
    df = pd.DataFrame(structured_data, columns=["Anker Ref.", "Descrizione", "Volume", "Quantity", "Price", "Total"])

    # Applica la formattazione dei valori per Price e Total
    df['Price'] = df['Price'].apply(formatta_valori)
    df['Total'] = df['Total'].apply(formatta_valori)
    df['Volume'] = df['Volume'].apply(lambda v: v.replace('.', ','))  # Sostituisci i punti con virgole nei volumi

    # 6. Salvare i dati in un file CSV nella cartella temporanea di Colab
    output_csv_path = '/content/ConvertedInvoice.csv'  # Salva il CSV in Colab
    df.to_csv(output_csv_path, index=False, sep=';')
    print(f"CSV salvato in: {output_csv_path}")

    # 7. Salvare i dati non processati in un file TXT nella cartella temporanea di Colab
    output_txt_path = '/content/dati_non_processati.txt'  # Salva il TXT in Colab
    with open(output_txt_path, 'w', encoding='utf-8') as f:
        for line in unstructured_data:
            f.write(line + '\n')

    # 8. Salvare i dati non riconosciuti in un file TXT
    output_txt_path = '/content/dati_non_riconosciuti.txt'  # Percorso per il file TXT
    with open(output_txt_path, 'w', encoding='utf-8') as f:
        for line in error_data:
            f.write(line + '\n')

    # 8. Cancella le immagini create durante il processo
    for image_path in image_paths:
        try:
            os.remove(image_path)
            # print(f"Immagine cancellata: {image_path}")
        except OSError as e:
            print(f"Errore nella cancellazione dell'immagine {image_path}: {e}")

    print(f"I dati sono stati salvati in: {output_csv_path}")
    print()
else:
    print("Nessun file PDF selezionato.")
    print()


CSV salvato in: /content/ConvertedInvoice.csv
I dati sono stati salvati in: /content/ConvertedInvoice.csv

